# С помощью этой методики можно найти свободные IMSI диапазоны, а также создать файл с ключами для отправки кастомеру

In [15]:
# Загрузить данные из файлов
# Задать имена аккаунтов и файлов
#----------------------------------

import pandas as pd
from pandas import DataFrame
from os.path import join, normpath
import datetime as dt

downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA\SoftProfiles\Data'
result_file = 'result_file.csv'
files = ['rb_42501_25_06_20181.txt',
         'rb_42501_2_09_20181.txt',
         'rb_26006_25_06_20181.txt',
         'rb_26006_2_09_20181.txt',
         'rb_26006_30_04_2019.txt',
         'rb_42501_30_04_2019.txt',]

reseller_name = 'Roamability Resellers Test'
s1_account_name = 'Tottolli S1 Soft'
s2_account_name = ''

customer_name = 'Tottolli' # to use as a file name

current_date = dt.datetime.now().strftime('%y%m%d')
file_s1_name = f'S1_{customer_name}_{current_date}.txt'
file_s2_name = f'S2_{customer_name}_{current_date}.txt'

df_imsi = DataFrame(columns = ['IMSI', 'KI', 'OPC'])

for file in files:
    df_imsi = df_imsi.append(pd.read_table(join(downloads, file), sep=' ',header=0, engine='python'))

df_imsi.IMSI = df_imsi.IMSI.astype('int64')
df_imsi.sort_values(by = 'IMSI', inplace = True)

df_imsi.head(3)

,IMSI,KI,OPC
0,260060140890000,1AEA5D122A1F2476EBD9350F10C0EFDD,5EE04A60E740AF9890C397A899109B25
1,260060140890001,09B595FF54EEFE9E547237B4D7499202,9274F199697E5775100367CAE65B24D0
2,260060140890002,744A3D38AFBCBB0292FF9807D21E8B4D,8D7B5100C7BF361382490BFA632F8A55


In [3]:
# (ОПЦИОНАЛЬНО)
# Предварительная проверка диапазонов (опционально)
#----------------------------------

list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]

df = df_imsi.groupby(df_imsi.IMSI.astype('str').str.slice(0,12))['IMSI'].agg(list_agg).reset_index()
df.IMSI = df.IMSI.astype('int64')
df.rename(columns = {'IMSI':'IMSI_RANGE'}, inplace = True)

df['imsi_shifted'] = df['IMSI_RANGE'].shift(-1)
df.fillna(0,inplace=True)
df['imsi_shifted'] = df['imsi_shifted'].astype('int64')

def check_imsi(x):
    x['imsi_num'] = x['IMSI_MAX'] - x['IMSI_MIN'] + 1
    x['check_range'] = x['imsi_shifted'] - x['IMSI_RANGE']
    if x['NUM'] == (x['IMSI_MAX'] - x['IMSI_MIN'] + 1):
        x['imsi_num_check'] = 'OK'
    else:
        x['imsi_num_check'] = 'NOK'
    return x

df = df.apply(check_imsi, axis=1)
df.head(3)

,IMSI_RANGE,IMSI_MIN,IMSI_MAX,NUM,imsi_shifted,imsi_num,check_range,imsi_num_check
0,260060140890,260060140890000,260060140890999,1000,260060140891,1000,1,OK
1,260060140891,260060140891000,260060140891999,1000,260060140892,1000,1,OK
2,260060140892,260060140892000,260060140892999,1000,260060140893,1000,1,OK


In [6]:
# (ОПЦИОНАЛЬНО)
df[df.IMSI_RANGE.astype('str').str.startswith('26006014998', na=False)].sort_values(by='IMSI_RANGE')

,IMSI_RANGE,IMSI_MIN,IMSI_MAX,NUM,imsi_shifted,imsi_num,check_range,imsi_num_check
10,260060149982,260060149982000,260060149982999,1000,260060149983,1000,1,OK
11,260060149983,260060149983000,260060149983999,1000,260060149984,1000,1,OK
12,260060149984,260060149984000,260060149984999,1000,260060149985,1000,1,OK
13,260060149985,260060149985000,260060149985999,1000,260060149986,1000,1,OK
14,260060149986,260060149986000,260060149986999,1000,260060149987,1000,1,OK
15,260060149987,260060149987000,260060149987999,1000,260060149988,1000,1,OK
16,260060149988,260060149988000,260060149988999,1000,260060149989,1000,1,OK
17,260060149989,260060149989000,260060149989999,1000,260060149991,1000,2,OK


In [4]:
# df.to_csv(join(downloads,'soft_ranges.csv'), index=False)

In [23]:
# Из базы данных OCS выгружаем IMSI
#----------------------------------

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT
r.RESELLER_NAME
,a.ACCOUNT_NAME
,ss.STATUS AS SUB_STATUS
,si.IMSI
FROM RESELLERS r
INNER JOIN ACCOUNTS a
ON r.RESELLER_ID = a.RESELLER_ID
INNER JOIN SUBSCRIBERS sr
ON sr.ACCOUNT_ID = a.ACCOUNT_ID
INNER JOIN SUBSCRIBER_STATUS ss
ON ss.SUBSCRIBER_ID = sr.SUBSCRIBER_ID
INNER JOIN SUBSCRIBER_IMSIS si
ON ss.SUBSCRIBER_ID = si.SUBSCRIBER_ID
WHERE
r.deleted is NULL
AND ss.END_DATE IS NULL
AND ss.status = 'Active'
AND
(si.IMSI BETWEEN '260060140890000' AND '260060140899999'
OR si.IMSI BETWEEN  '260060149991100' AND '260060149999999'
OR si.IMSI BETWEEN  '425019613998060' AND '425019613999999'
OR si.IMSI BETWEEN  '425019629920000' AND '425019629939999'
OR si.IMSI BETWEEN  '425019613790000' AND '425019613799999'
OR si.IMSI BETWEEN  '425019613788000' AND '425019613789999'
OR si.IMSI BETWEEN  '260060149982000' AND '260060149989999')
'''

df_ocs_imsi = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_ocs_imsi['IMSI'] = df_ocs_imsi['IMSI'].astype('int64')
cnxn.close()
df_ocs_imsi.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI
0,Acloud RT,Acloud Test,Active,260060149991108
1,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991111
2,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991112


In [24]:
# Объединяем IMSI с ключами с данными OCS
#----------------------------------

df_ocs_imsi_ki = pd.merge(df_imsi, df_ocs_imsi, how='left', on='IMSI')
list_agg = [('IMSI_MIN','min'),('IMSI_MAX','max'),('NUM','count')]
list_col = ['RESELLER_NAME', 'ACCOUNT_NAME', 'SUB_STATUS', df_ocs_imsi_ki.IMSI.astype('str').str.slice(0,12)]
df_ocs_imsi_ki_group = df_ocs_imsi_ki.groupby(list_col)['IMSI'].agg(list_agg).reset_index().sort_values(by='IMSI')

df_ocs_imsi_ki_group.head(3)

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
17,MCN Telecom,MCN Telecom Soft SIM,Active,260060140890,260060140890000,260060140890999,1000
18,MCN Telecom,MCN Telecom Soft SIM,Active,260060140891,260060140891000,260060140891999,1000
19,MCN Telecom,MCN Telecom Soft SIM,Active,260060140892,260060140892000,260060140892999,1000


In [18]:
# For test accounts to use the following ranges:
# S1: 425019613998*
# S2: 260060149991*
# Выбрать свободные из "Roamability Invertory"

s1_test_soft_range = '425019613998'
s2_test_soft_range = '260060149991'

print('S1 test IMSIs in OCS by Reseller:')
display(df_ocs_imsi_ki_group[df_ocs_imsi_ki_group['IMSI'] == s1_test_soft_range].sort_values('IMSI_MIN'))

print('S2 test IMSIs in OCS by Reseller:')
display(df_ocs_imsi_ki_group[df_ocs_imsi_ki_group['IMSI'] == s2_test_soft_range].sort_values('IMSI_MIN'))

S1 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
8,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,425019613998,425019613998060,425019613998064,5
23,MCN Telecom,MCN Telecom Soft SIM,Active,425019613998,425019613998065,425019613998067,3
1,Acloud RT,Acloud Test,Active,425019613998,425019613998068,425019613998069,2
80,XOtel,XOtel Test account,Active,425019613998,425019613998070,425019613998070,1
66,Tottolli-Tech-Mobil,Test Sim cards,Active,425019613998,425019613998071,425019613998081,4
3,Dialoq - Genesisclub RT,Genesisclub Partner Soft test,Active,425019613998,425019613998072,425019613998937,18
64,Tapmobile RT,Tapmobile Test Sim cards,Active,425019613998,425019613998078,425019613998078,1
12,Joy_telecom,Joy telecom Partner Soft test,Active,425019613998,425019613998088,425019613998092,4
77,Watchdata SG,Watchdata Partner Soft test,Active,425019613998,425019613998090,425019613998090,1
38,RedTea RT,RedTea TEST Soft-simcards account,Active,425019613998,425019613998093,425019613998112,20


S2 test IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
7,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,260060149991,260060149991100,260060149991104,5
22,MCN Telecom,MCN Telecom Soft SIM,Active,260060149991,260060149991105,260060149991107,3
0,Acloud RT,Acloud Test,Active,260060149991,260060149991108,260060149991109,2
2,Dialoq - Genesisclub RT,Genesisclub P4 Soft test,Active,260060149991,260060149991110,260060149991945,18
63,Tapmobile RT,Tapmobile Test Sim cards,Active,260060149991,260060149991116,260060149991116,1
78,Watchdata SG,Watchdata P4 Soft test,Active,260060149991,260060149991123,260060149991123,1
39,Roamability Invertory,P4 Soft simcards inventory,Active,260060149991,260060149991124,260060149991939,816
56,Roamability Resellers Test,Roamwifi S2,Active,260060149991,260060149991946,260060149991947,2
54,Roamability Resellers Test,DrimSim S2,Active,260060149991,260060149991948,260060149991949,2
30,MCN Telecom,MCN Telecom Soft SIM S2,Active,260060149991,260060149991950,260060149991954,5


In [25]:
# For commersial usage

# S1 profiles
print('S1 prod IMSIs in OCS by Reseller:')
display(df_ocs_imsi_ki_group[df_ocs_imsi_ki_group.IMSI.str.startswith('42501', na=False)])

# S2 profiles
print('S2 prod IMSIs in OCS by Reseller:')
display(df_ocs_imsi_ki_group[df_ocs_imsi_ki_group.IMSI.str.startswith('26006', na=False)])

S1 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
15,Justworks Mobile,Justworks S1 test soft SIM,Active,425019613998,425019613998995,425019613998995,1
58,Roamability Resellers Test,VTB S1 soft,Active,425019613998,425019613998991,425019613998994,4
60,Simlocal,Test SIM cards,Active,425019613998,425019613998996,425019613998996,1
12,Joy_telecom,Joy telecom Partner Soft test,Active,425019613998,425019613998088,425019613998092,4
62,Taisys,Test Sim cards,Active,425019613998,425019613998997,425019613998999,3
64,Tapmobile RT,Tapmobile Test Sim cards,Active,425019613998,425019613998078,425019613998078,1
66,Tottolli-Tech-Mobil,Test Sim cards,Active,425019613998,425019613998071,425019613998081,4
8,Flexiroam Limited -- TESTING at 2018-07,Flexiroam Test Soft simcard,Active,425019613998,425019613998060,425019613998064,5
6,Drimsim,Drimsim S1 Soft test IMSI,Active,425019613998,425019613998950,425019613998984,15
4,Dialoq - Genesisclub RT,Truphone S1 Soft,Active,425019613998,425019613998960,425019613998989,25


S2 prod IMSIs in OCS by Reseller:


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI,IMSI_MIN,IMSI_MAX,NUM
17,MCN Telecom,MCN Telecom Soft SIM,Active,260060140890,260060140890000,260060140890999,1000
18,MCN Telecom,MCN Telecom Soft SIM,Active,260060140891,260060140891000,260060140891999,1000
19,MCN Telecom,MCN Telecom Soft SIM,Active,260060140892,260060140892000,260060140892999,1000
20,MCN Telecom,MCN Telecom Soft SIM,Active,260060140893,260060140893000,260060140893999,995
21,MCN Telecom,MCN Telecom Soft SIM,Active,260060140894,260060140894000,260060140894999,1000
72,Tottolli-Tech-Mobil,Tottolli S2 Soft,Active,260060140895,260060140895000,260060140895999,1000
73,Tottolli-Tech-Mobil,Tottolli S2 Soft,Active,260060140896,260060140896000,260060140896999,1000
74,Tottolli-Tech-Mobil,Tottolli S2 Soft,Active,260060140897,260060140897000,260060140897999,1000
75,Tottolli-Tech-Mobil,Tottolli S2 Soft,Active,260060140898,260060140898000,260060140898999,1000
76,Tottolli-Tech-Mobil,Tottolli S2 Soft,Active,260060140899,260060140899000,260060140899999,1000


In [20]:
# Записать выбранные IMSI for test accounts
# Проверить выбранные IMSI на принадлежность к "Roamability Invertory"

s1_imsis = ['425019613998990']
s2_imsis = ['260060149991992']

display(df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').isin(s1_imsis)].sort_values('IMSI'))
display(df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').isin(s2_imsis)].sort_values('IMSI'))

,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI
35770,Roamability Resellers Test,Clementvale S1 soft,Active,425019613998990


,RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,IMSI
20206,Roamability Resellers Test,Clementvale S2 soft,Active,260060149991992


In [26]:
# Записать выбранные диапазоны IMSI (для коммерческих аккаунтов)
# Проверить выбранные IMSI на принадлежность к "Roamability Invertory"


s1_imsi_range_start = 425019629935000
s1_imsi_range_end = 425019629935999

s2_imsi_range_start = 260060000000000
s2_imsi_range_end = 260060000000000

s1_imsis = [str(imsi) for imsi in range(s1_imsi_range_start,s1_imsi_range_end+1,1)]
s2_imsis = [str(imsi) for imsi in range(s2_imsi_range_start,s2_imsi_range_end+1,1)]

df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').isin(s1_imsis)].sort_values('IMSI')
display(df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').isin(s1_imsis)].groupby(['RESELLER_NAME',
                                                                            'ACCOUNT_NAME','SUB_STATUS']).count())
display(df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').isin(s2_imsis)].groupby(['RESELLER_NAME',
                                                                            'ACCOUNT_NAME','SUB_STATUS']).count())

,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,
Roamability Resellers Test,Tottolli S1 Soft,Active,1000


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


In [22]:
# Создать Soft аккаунты для Реселлера и перенести туда IMSI.

print(f'Reseller: {reseller_name}\nAccount S1: {s1_account_name}\nAccount S2: {s2_account_name}')

Reseller: Roamability Resellers Test
Account S1: Tottolli S1 Soft
Account S2: 


In [27]:
# После переноса IMSI в аккаунты проверить выбранные IMSI и диапазоны
# (предварительно перезагрузив данные из OCS)

# Для диапазонов IMSI
display(df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').isin(s1_imsis)].groupby(['RESELLER_NAME','ACCOUNT_NAME','SUB_STATUS']).count())
display(df_ocs_imsi[df_ocs_imsi.IMSI.astype('str').isin(s2_imsis)].groupby(['RESELLER_NAME','ACCOUNT_NAME','SUB_STATUS']).count())

,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,
Roamability Resellers Test,Tottolli S1 Soft,Active,1000


,,,IMSI
RESELLER_NAME,ACCOUNT_NAME,SUB_STATUS,


In [28]:
# Выгружаем файлы с ключами для диапазонов IMSI
#----------------------------------

if len(s1_account_name) != 0:
    df_ocs_imsi_ki.loc[(df_ocs_imsi_ki.RESELLER_NAME == reseller_name) & (df_ocs_imsi_ki.ACCOUNT_NAME == s1_account_name)
                       & (df_ocs_imsi_ki.IMSI.astype('str').isin(s1_imsis)),
                   ['IMSI', 'KI', 'OPC']].to_csv(join(downloads, file_s1_name), index=False)

if len(s2_account_name) != 0:
    df_ocs_imsi_ki.loc[(df_ocs_imsi_ki.RESELLER_NAME == reseller_name) & (df_ocs_imsi_ki.ACCOUNT_NAME == s2_account_name)
                       & (df_ocs_imsi_ki.IMSI.astype('str').isin(s2_imsis)),
                   ['IMSI', 'KI', 'OPC']].to_csv(join(downloads, file_s2_name), index=False)

### Далее шифруем и отправляем
----------------------------------
1. Упаковать оба файла в zip архив.
2. Выполнить экспорт открытого ключа Кастомера (дважды кликнуть по файлу ключа).
3. В папке с архивом по пр.кл. на zip архив и выбрать "Зашифровать"
4. Зашифровать своим ключем *_FOR_KEYS
5. Отправить кастомеру. Свой ключ отправлять не нужно.